<a href="https://colab.research.google.com/github/ijusplab/criar-resumo/blob/main/Criar_Resumo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **10ª Turma Recursal**
## Gerenciamento de Pauta

## **Instruções**

**INSTRUÇÕES**

1) Execute a célula `CARREGAR FERRAMENTA`. 

2) Execute a célula `CARREGAR DADOS DO SHAREPOINT`. 

3) Assim que o código for inicializado e aparecer o botão de `upload`, carregue o arquivo `csv` do sharepoint. Você verá uma prévia do conteúdo extraído logo abaixo da célula, em forma de tabela.

4) Execute a célula `GERAR ARQUIVO JSON`. O arquivo será baixado no seu browser em poucos segundos.


## **CARREGAR FERRAMENTA**

In [ ]:
#@title **CARREGAR FERRAMENTA** { vertical-output: true }
MODO_DEPURACAO = False

import re, os, json, csv, pprint
from google.colab import files
import pandas as pd
import numpy as np
from IPython.display import display, HTML

##
## Mensagens
##

class mensageria:
  __END      = '\33[0m'
  __BOLD     = '\33[1m'
  __ITALIC   = '\33[3m'
  __URL      = '\33[4m'
  __BLINK    = '\33[5m'
  __BLINK2   = '\33[6m'
  __SELECTED = '\33[7m'

  __BLACK  = '\33[30m'
  __RED    = '\33[31m'
  __GREEN  = '\33[32m'
  __YELLOW = '\33[33m'
  __BLUE   = '\33[34m'
  __VIOLET = '\33[35m'
  __BEIGE  = '\33[36m'
  __WHITE  = '\33[37m'

  __BLACKBG  = '\33[40m'
  __REDBG    = '\33[41m'
  __GREENBG  = '\33[42m'
  __YELLOWBG = '\33[43m'
  __BLUEBG   = '\33[44m'
  __VIOLETBG = '\33[45m'
  __BEIGEBG  = '\33[46m'
  __WHITEBG  = '\33[47m'

  __GREY    = '\33[90m'
  __RED2    = '\33[91m'
  __GREEN2  = '\33[92m'
  __YELLOW2 = '\33[93m'
  __BLUE2   = '\33[94m'
  __VIOLET2 = '\33[95m'
  __BEIGE2  = '\33[96m'
  __WHITE2  = '\33[97m'

  __GREYBG    = '\33[100m'
  __REDBG2    = '\33[101m'
  __GREENBG2  = '\33[102m'
  __YELLOWBG2 = '\33[103m'
  __BLUEBG2   = '\33[104m'
  __VIOLETBG2 = '\33[105m'
  __BEIGEBG2  = '\33[106m'
  __WHITEBG2  = '\33[107m'

  @classmethod
  def sucesso(cls, msg):
    print(f'{cls.__BLUE}{cls.__BOLD}{msg}{cls.__END}')

  @classmethod
  def falha(cls, msg):
    print(f'{cls.__REDBG}{cls.__WHITE}{msg}{cls.__END}')

  @classmethod
  def alerta(cls, msg):
    print(f'{cls.__YELLOWBG}{msg}{cls.__END}')

  @classmethod
  def ok(cls, msg):
    print(f'{cls.__GREEN}{cls.__BOLD}{msg}{cls.__END}')

  @classmethod
  def normal(cls, msg):
    print(f'{cls.__BOLD}{msg}{cls.__END}')

  @classmethod
  def mostrar_objeto(cls, obj):
    printer = pprint.PrettyPrinter(indent=4)
    printer.pprint(obj)

##
## Para normalização dos nomes de colunas
##
class utils:
  __acentos = {
    'a': re.compile('[áàäãâ]'), 
    'e': re.compile('[éèëê]'), 
    'i': re.compile('[íìïî]'),
    'o': re.compile('[óòöõô]'),
    'u': re.compile('[úùüû]'),
    'c': re.compile('[ç]'),
    'A': re.compile('[ÁÀÄÃÂ]'), 
    'E': re.compile('[ÉÈËÊ]'), 
    'I': re.compile('[ÍÌÏÎ]'),
    'O': re.compile('[ÓÒÖÕÔ]'),
    'U': re.compile('[ÚÙÜÛ]'),
    'C': re.compile('[Ç]')
  }
  __caracteres_especiais = re.compile('[\(\)\[\]\{\}!@#$%¨&*+=ªº°§]')

  @classmethod
  def remover_acentuacao(cls, s):
    for letra, padrao in cls.__acentos.items():
      s = re.sub(padrao, letra, s)
    return s

  @classmethod
  def limpar(cls, s):
    s = cls.remover_acentuacao(s).upper().strip()
    s = re.sub(cls.__caracteres_especiais, '', s).strip()
    s = re.sub(r'\s{2,}', ' ', s).strip()
    return s

  @classmethod
  def normalizar(cls, s):
    s = cls.remover_acentuacao(s).upper().strip()
    s = re.sub(r'\d+_+', '', s)
    s = re.sub(r'\s+', '_', s)
    return s

##
## Extração dos dados do sharepoint
##
class Sharepoint:
  _instance = None

  def __init__(self):
    self.__resumo = None
    self.__votos = None
    self.__pastas = None
    self.__nome = ''
    self._cols = {
      "PROCESSO": "PROCESSO",
      "OBJETO": "OBJETO",
      "ALEGACOES": "ALEGACOES",
      "FUNDAMENTACAO": "FUNDAMENTACAO",
      "DISPOSITIVO": "DISPOSITIVO",
      "OBSERVACOES": "OBSERVACOES",
      "STATUS_JULGAMENTO": "STATUS_JULGAMENTO",
      "SITUACAO_PAUTA": "SITUACAO_PAUTA",
      "TIPO_VOTO": "TIPO_VOTO",
      "{IsFolder}": "IS_FOLDER",
      "{Link}": "LINK",
      "{Name}": "NAME",
      "{Path}": "PATH",
      "{FullPath}": "FULL_PATH"
    }

  @classmethod
  def instance(cls):
      if cls._instance is None:
          cls._instance = cls()
      return cls._instance

  def __extrair_cadeira(self, s):
    pattern = re.compile('Cadeira\s(\d+)')
    match = pattern.search(s)
    if not match:
      return ''
    return match.group(1)

  def __transformar_nome_coluna(self, s):
    if s in self._cols:
      return self._cols[s]
    return s

  def __extrair_valor(self, s):
    pattern = re.compile('\"Value\":\"(.+)\"')
    match = pattern.search(s)
    if not match:
      return ''
    return match.group(1)

  def __extrair_numero_processo(self, s):
    pattern = re.compile('\d{7}-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4}')
    match = pattern.search(s)
    if not match:
      return ''
    return match.group(0)

  # obtem grupo do voto (a partir das duas pastas acima)
  # a partir de /pasta1/pasta2/arquivo, retorna pasta1_pasta2 
  def __obter_grupo(self, s):
    pattern = re.compile('.*\/([^\/]+\/[^\/]+)\/')
    match = pattern.search(s)
    if not match:
      return ''
    return match.group(1).replace('/', '_')

  def __obter_dic_processos(self, df):
    dic = {}
    for index, row in df.iterrows():
      key = re.sub('\D', '', row['PROCESSO'])
      dic[key] = {}
      dic[key]['so'] = False
      dic[key]['adv'] = ''
      dic[key]['sit'] = row['SITUACAO_PAUTA']
      dic[key]['concl'] = False
      dic[key]['jimp'] = ''
    return dic

  def __obter_grupos_ordenados(self, df_folders):
    list = []
    pattern = re.compile('^\d+$')
    for index, row in df_folders.iterrows():
      dic = {}
      dic['titulo'] = row['GRUPO']
      dic['status'] = re.sub('\D', '', row['STATUS_JULGAMENTO'])
      dic['situacao'] = re.sub('\D', '', row['SITUACAO_PAUTA'])
      dic['tipo'] = re.sub('\D', '', row['TIPO_VOTO'])
      dic['observacoes'] = row['OBSERVACOES']
      dic['dispositivo'] = row['DISPOSITIVO']
      dic['link'] = row['LINK']
      lote = len(row['LOTE'].split('\n'))
      dic['lote'] = [] if len(row['LOTE']) == 0 else lote
      list.append(dic)
    list.sort(key=lambda item: 1000 if not pattern.search(item['tipo']) else int(item['tipo']))
    return list

  def __obter_list_resumo(self, df_files, sorted_groups):
    grouped = df_files.groupby('GRUPO')
    resumo = []
    counter = 1

    for group in sorted_groups:
      try:
        items = grouped.get_group(group['titulo'])
      except KeyError:
        continue
      if len(items) == 0:
        continue

      root = {}
      titulo = group['titulo']
      root['titulo'] = f'{counter}.{titulo}'
      root['observacoes'] = group['observacoes']
      root['link'] = group['link']
      root['tipo'] = 'lote' if len(group['lote']) > 0 else 'normal'
      coluna1 = {}
      coluna1['numero'] = 1
      coluna1['tamanho'] = 47
      coluna1['nome'] = 'Alegações Recursais'
      coluna2 = {}
      coluna2['numero'] = 2
      coluna2['tamanho'] = 35
      coluna2['nome'] = 'Fundamentação'
      coluna3 = {}
      coluna3['numero'] = 2
      coluna3['tamanho'] = 18
      coluna3['nome'] = 'Resultado'
      root['colunas'] = [coluna1, coluna2, coluna3]

      processos = []
      subcounter = 1
      for index, row in items.iterrows():
        processo = {}
        processo['numero'] = re.sub('\D', '', row['PROCESSO'])
        processo['item'] = f'{counter}.{subcounter}'
        processo['ordem']: f'{str(counter).zfill(3)}{str(index).zfill(3)}'
        coluna1 = {}
        coluna1['numero'] = 1
        coluna1['tamanho'] = 47
        coluna1['conteudo'] = ''
        coluna2 = {}
        coluna2['numero'] = 2
        coluna2['tamanho'] = 35
        coluna2['conteudo'] = ''
        coluna3 = {}
        coluna3['numero'] = 2
        coluna3['tamanho'] = 18
        coluna3['conteudo'] = row['DISPOSITIVO']
        processo['colunas'] = [coluna1, coluna2, coluna3]
        processos.append(processo)
        subcounter += 1

      root['processos'] = processos
      resumo.append(root)
      counter += 1
    
    return resumo

  def load(self):
    # uploaded = files.upload()
    # arquivos = list(uploaded.keys())
    # print(arquivos)
    # if len(arquivos) > 0:
    nome = '2021-07-30.csv' #arquivos[0]
    # conteudo = uploaded[nome] 

    if re.match('^.+\.(csv)$', nome):

      # converte csv em dataframe, mantendo somente as
      # colunas desejadas
      df = pd.read_csv(nome, sep=',', encoding='utf-8')
      df = df.fillna('')
      df.drop(df.columns.difference(list(self._cols.keys())), 1, inplace=True)
      df.columns = map(self.__transformar_nome_coluna, df.columns)

      # Compõe o nome do arquivo
      cadeira = self.__extrair_cadeira(df.at[0, 'PATH'])
      data = nome.replace('.csv', '').split('-')
      data.reverse()
      data = ''.join(data)
      self.__nome = f'{cadeira}{data}'

      # Extrai o valor das colunas customizadas do Sharepoint
      custom = ['STATUS_JULGAMENTO', 'SITUACAO_PAUTA', 'TIPO_VOTO', 'OBSERVACOES', 'DISPOSITIVO', 'LOTE']
      for col_name in custom:
        if col_name not in df.columns:
          df[col_name] = pd.Series()
          df[col_name] = df[col_name].fillna('')
        else:
          df[col_name] = df[col_name].apply(self.__extrair_valor)

      # Separa pastas e arquivos
      grouped = df.groupby(df['IS_FOLDER'])
      folders = grouped.get_group(True).copy()
      files = grouped.get_group(False).copy()

      # Identifica as divisões do resumo a partir das pastas
      folders['GRUPO'] = folders['FULL_PATH'].apply(lambda x: f'{x}/').apply(self.__obter_grupo)
      grupos = self.__obter_grupos_ordenados(folders)

      # cria coluna com números de processos
      # e identifica se há nomes de arquivos inválidos
      files['PROCESSO'] = files['NAME'].apply(self.__extrair_numero_processo)
      try:
        errors = files.groupby(files['PROCESSO']).get_group('')['NAME'].tolist()
      except:
        errors = []

      if len(errors) > 0:
        errors = '\n'.join(errors)
        mensageria.falha(f'\n>>>> Os seguintes arquivos têm nome irregular:\n {errors}')
        return

      # cria coluna com a localização dos processos
      # e identifica se há localizações inválidas
      files['GRUPO'] = files['PATH'].apply(self.__obter_grupo)

      try:
        errors = files.groupby(files['GRUPO']).get_group('')['NAME'].tolist()
      except:
        errors = []

      if len(errors) > 0:
        errors = '\n'.join(errors)
        mensageria.falha(f'\n>>>> Os seguintes arquivos têm localização irregular:\n {errors}')
        return

      # Converte dados em dicionário
      self.__resumo = {}
      self.__resumo['processos'] = self.__obter_dic_processos(files)
      self.__resumo['resumo'] = self.__obter_list_resumo(files, grupos)
      self.__votos = files
      self.__pastas = folders

      # os.remove(nome)
      # else:
      #   mensageria.falha(f'\n>>>> O arquivo {nome} não é um txt')

  def has_data_frame(self):
    return not self.__votos is None and not self.__pastas is None

  def get_name(self):
    if not self.has_data_frame():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    return self.__nome

  def get_data_frame(self):
    if not self.has_data_frame():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    return self.__data_frame

  def show_votos(self):
    if not self.has_data_frame():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    print()
    mensageria.normal('\n>>>> VOTOS')
    display(HTML(self.__votos.head().to_html()))
    mensageria.normal('\n>>>> TIPOS')
    for col in self.__votos.columns:
      print(f'{col}: {self.__votos[col].dtype}')

  def show_pastas(self):
    if not self.has_data_frame():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    print()
    mensageria.normal('\n>>>> PASTAS')
    display(HTML(self.__pastas.head().to_html()))
    mensageria.normal('\n>>>> TIPOS')
    for col in self.__pastas.columns:
      print(f'{col}: {self.__pastas[col].dtype}')

  def show_resumo(self):
    if not self.has_data_frame():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    mensageria.normal('\n>>>> JSON')
    print(json.dumps(self.__resumo, indent=2))

  def download(self):
    if not self.has_data_frame():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    nome = f'{self.__nome}.json'
    if os.path.isfile(nome):
      os.remove(nome)
    with open(nome, 'w', encoding='utf-8') as file:
      file.write(json.dumps(self.__resumo, indent=2))
    files.download(nome)

print()
mensageria.sucesso('\n>>>> Ferramentas carregadas com sucesso!')

In [ ]:
#@title **CARREGAR DADOS DO SHAREPOINT** { vertical-output: true }

##
## Instanciação
##
sharepoint = Sharepoint.instance()
sharepoint.load()
sharepoint.show_pastas()
sharepoint.show_votos()
sharepoint.show_resumo()

In [ ]:
#@title **GERAR ARQUIVO `JSON`**

sharepoint.download()